<a href="https://colab.research.google.com/github/AgustinBustos/no_more_exponential/blob/main/equDiff_nn_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Differential Equations With NN

In [1]:
import torch
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

This is an introduction and first exploration of a recent idea i had while working with the ARIMA model, as always im sure its done somewhere, but maybe differential equations are a bit overvalued (especially in the realm of economics), so my sense is that its going to take some time in order for me to really use the results capabilities. <br/>

We are going to work with the classic equation: $$ \dot{f}=f$$<br/>
giving the result: <br/> $$f=ae^x$$ 

In [2]:
x=np.linspace(-2,2)
y=np.exp(x)
px.line(x=x,y=y).show()

#Functions
Lets define some functions for later

In [3]:
def mse(t1,t2):
  diff = t1 - t2
  return torch.sum(diff * diff) / diff.numel()
  
sig= torch.nn.Sigmoid()
relu = torch.nn.ReLU()
tan = torch.nn.Tanh()

# Main

We are going to have a simple neural network with 1 neuron as input, 100 as hidden layer and 1 as output, so lets define the input xt:

In [4]:
xt=torch.linspace(-2,2,50,requires_grad=True)

Now lets define the weights

In [5]:
measure=100
w=torch.rand([1,measure],requires_grad=True) # ojo lo obligo a ser creaciente en la inicialiacion rand
b=torch.rand(measure,requires_grad=True)

w1=torch.rand([measure,1],requires_grad=True)
b1=torch.rand(1,requires_grad=True)

The graph has the exponential function and the network with random weights

In [6]:
def f_hat(x):
  layer1=sig(torch.reshape(x,(-1,1)) @ w+b)  #relu o sig  #b #tan
  result=layer1 @ w1 + b1  #b1
  return result.reshape(-1) 


fig1=px.line(x=xt.reshape(-1).detach().numpy(),y=f_hat(xt).detach().numpy())
fig2=px.line(x=x,y=y)
fig3 = go.Figure(data=fig1.data + fig2.data)
fig3.show()

# Main Function, df_hat/dx

This is the most important function, using the pytorch library, its trivial to get the derivative of the network. Its crucial to create the graph for the derivative so that the weights take into account both the movement of the function and the movement of the derivative.

In [7]:

def f_hat_prime(x):
  return torch.autograd.grad(f_hat(x), x,grad_outputs=torch.ones_like(f_hat(x)), create_graph=True)[0]
f_hat_prime(xt)



tensor([5.4685, 5.5701, 5.6681, 5.7619, 5.8511, 5.9350, 6.0130, 6.0846, 6.1493,
        6.2066, 6.2560, 6.2970, 6.3295, 6.3529, 6.3671, 6.3720, 6.3673, 6.3531,
        6.3295, 6.2965, 6.2543, 6.2032, 6.1435, 6.0757, 6.0001, 5.9173, 5.8277,
        5.7320, 5.6306, 5.5243, 5.4136, 5.2991, 5.1814, 5.0612, 4.9389, 4.8151,
        4.6904, 4.5652, 4.4400, 4.3152, 4.1912, 4.0683, 3.9469, 3.8272, 3.7096,
        3.5941, 3.4810, 3.3704, 3.2624, 3.1572],
       grad_fn=<ReshapeAliasBackward0>)

The network and its derivative

In [8]:

px.line(x=list(xt.reshape(-1).detach().numpy())+list(xt.reshape(-1).detach().numpy()),
        y=list(f_hat(xt).detach().numpy())+list(f_hat_prime(xt).detach().numpy())).show()

This is the phase graph, we are going to try to get the points over the 45 degree line (df/dx=f).

In [9]:
fig1=px.scatter(x=list(f_hat_prime(xt).detach().numpy()),y=list(f_hat(xt).detach().numpy()))
fig2=px.line(x=np.linspace(0,7),y=np.linspace(0,7))

fig3 = go.Figure(data=fig1.data + fig2.data)
fig3.show()

# Main Idea

We have the equation: $$\dot{f}=f$$ We are going to use a network to try and aproximate the solution, so in the end it wont be possible to get an exact equality: $$\dot{\hat{f}}\approx\hat{f}\Longrightarrow\dot{\hat{f}}-\hat{f}=error$$
 And thats the trick, we will have an error of the type: $$\int\left(\dot{\hat{f}}-\hat{f}\right)^2=mse$$
Thats the main idea, analog to linear regression, we can transform a system of equations into an optimization problem and then use neural networks to solve it using gradient descent. <br/> In this case, the real loss is:

```
loss = mse(f_hat(xt), f_hat_prime(xt))
```




In [14]:
measure=100 
w=torch.rand([1,measure],requires_grad=True)
b=torch.rand(measure,requires_grad=True)

w1=torch.rand([measure,1],requires_grad=True)
b1=torch.rand(1,requires_grad=True)


optimizer = torch.optim.Adam([w,b,w1,b1], lr=0.1) 


for epoch in range(10000):   
    loss = mse(f_hat(xt), f_hat_prime(xt))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
fig1=px.scatter(x=list(f_hat_prime(xt).detach().numpy()),y=list(f_hat(xt).detach().numpy()))
a=f_hat(torch.tensor(0.))[0].item()
fig2=px.line(x=np.linspace(0,a*2.71**2),y=np.linspace(0,a*2.71**2))

fig3 = go.Figure(data=fig1.data + fig2.data)
fig3.show()


f_hat againts its derivative

In [15]:
px.line(x=list(xt.reshape(-1).detach().numpy())+list(xt.reshape(-1).detach().numpy()),
        y=list(f_hat(xt).detach().numpy())+list(f_hat_prime(xt).detach().numpy())).show()

f_hat against the exponential

In [16]:
fig1=px.line(x=xt.reshape(-1).detach().numpy(),y=f_hat(xt).detach().numpy())
a=f_hat(torch.tensor(0.))[0].item()
fig2=px.line(x=x,y=y*a)
fig3 = go.Figure(data=fig1.data + fig2.data)
fig3.show()



# check

In [51]:
yy=2*xt
yy=yy.detach()

In [15]:
f_hat_prime(xt)

tensor([5.3881, 5.5071, 5.6230, 5.7353, 5.8434, 5.9467, 6.0445, 6.1361, 6.2211,
        6.2987, 6.3684, 6.4298, 6.4822, 6.5252, 6.5585, 6.5818, 6.5947, 6.5971,
        6.5890, 6.5703, 6.5410, 6.5015, 6.4518, 6.3923, 6.3234, 6.2455, 6.1593,
        6.0652, 5.9638, 5.8559, 5.7420, 5.6229, 5.4992, 5.3717, 5.2411, 5.1079,
        4.9728, 4.8365, 4.6995, 4.5623, 4.4254, 4.2894, 4.1546, 4.0214, 3.8901,
        3.7610, 3.6344, 3.5104, 3.3894, 3.2713])

In [59]:
xt=torch.autograd.Variable(torch.linspace(-2,2,50), requires_grad=True)
xt=torch.linspace(-2,2,50,requires_grad=True)
def f_hat_prime(x):
  f_hat(x).backward(torch.ones_like(f_hat(x)))
  x.grad.zero_()

  f_hat(x).backward(torch.ones_like(f_hat(x)))  #,create_graph=True
  return x.grad

f_hat_prime(xt)

tensor([4.9906, 5.0840, 5.1743, 5.2609, 5.3434, 5.4213, 5.4943, 5.5617, 5.6233,
        5.6785, 5.7271, 5.7686, 5.8028, 5.8294, 5.8481, 5.8587, 5.8613, 5.8557,
        5.8418, 5.8198, 5.7898, 5.7519, 5.7063, 5.6533, 5.5933, 5.5265, 5.4534,
        5.3744, 5.2900, 5.2005, 5.1066, 5.0087, 4.9072, 4.8027, 4.6957, 4.5866,
        4.4759, 4.3640, 4.2514, 4.1384, 4.0254, 3.9127, 3.8007, 3.6897, 3.5798,
        3.4714, 3.3646, 3.2596, 3.1566, 3.0557])

In [67]:
#version mejorada
def f_hat_prime(x):
  return torch.autograd.grad(f_hat(x), x,grad_outputs=torch.ones_like(f_hat(x)), create_graph=True)[0]
f_hat_prime(xt)

tensor([-3.9534, -3.7976, -3.6376, -3.4756, -3.3128, -3.1501, -2.9880, -2.8266,
        -2.6659, -2.5055, -2.3452, -2.1848, -2.0240, -1.8626, -1.7006, -1.5380,
        -1.3750, -1.2116, -1.0480, -0.8845, -0.7212, -0.5583, -0.3956, -0.2334,
        -0.0714,  0.0905,  0.2524,  0.4145,  0.5770,  0.7400,  0.9036,  1.0676,
         1.2321,  1.3969,  1.5617,  1.7266,  1.8912,  2.0555,  2.2195,  2.3832,
         2.5468,  2.7104,  2.8741,  3.0380,  3.2020,  3.3658,  3.5290,  3.6907,
         3.8500,  4.0057], grad_fn=<ReshapeAliasBackward0>)

In [68]:
measure=100 
w=torch.rand([1,measure],requires_grad=True)
b=torch.rand(measure,requires_grad=True)

w1=torch.rand([measure,1],requires_grad=True)
b1=torch.rand(1,requires_grad=True)


optimizer = torch.optim.Adam([w,b,w1,b1], lr=0.1) 


for epoch in range(10000):   
    loss = mse(yy, f_hat_prime(xt))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
px.line(x=list(xt.reshape(-1).detach().numpy())+list(xt.reshape(-1).detach().numpy()),
        y=list(f_hat(xt).detach().numpy())+list(f_hat_prime(xt).detach().numpy())).show()